In [ ]:
# 必要なライブラリのインポート
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

# 指定URL
base_url = "https://www.jalan.net/kankou/spt_10426cd2110076558/kuchikomi/page_{}/?screenId=OUW2202&reviewRefineCompanion=all&reviewRefineMonth=all&resultSort=pd"

# ヘッダー設定
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# 結果を格納するリスト
visit_dates = []

# 最大ページ数
max_pages = 334

# スクレイピングのメイン処理
for page in range(1, max_pages + 1):
    # ページURLを作成
    url = base_url.format(page)
    print(f"[INFO] Scraping page {page}: {url}")

    try:
        # ページを取得
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # HTTPエラーがあれば例外をスロー
        soup = BeautifulSoup(response.content, "html.parser")

        # 各口コミブロックを取得
        review_blocks = soup.find_all("div", class_="item-reviewText")
        if not review_blocks:
            print(f"No reviews found on page {page}. Stopping...")
            break

        # 各ページの口コミを取得（333ページ目は2件、それ以外のページは10件）
        num_reviews_to_get = 10 if page < 333 else 2
        for review in review_blocks[:num_reviews_to_get]:

            # 訪問時期を取得
            details_element = review.find_next_sibling("ul", class_="item-reviewDetail")
            visit_date = "-"
            if details_element:
                detail_items = details_element.find_all("li")
                if len(detail_items) > 0:
                    visit_date = detail_items[0].text.replace("行った時期：", "").strip()

            # データをリストに追加
            visit_dates.append({"VisitDate": visit_date})

            # 各口コミをログとして表示
            print(f" VisitDate: {visit_date}")

        # リクエストごとにランダムな待機時間を設定
        wait_time = random.uniform(3, 5)
        print(f"Waiting for {wait_time:.2f} seconds before the next request...")
        time.sleep(wait_time)

    except requests.exceptions.RequestException as e:
        # エラー発生時のログ
        print(f"Failed to scrape page {page}: {e}")
        continue

# 結果の確認（取得した訪問時期の表示、一部のみ）
print(f"Scraping completed. Total visit dates scraped: {len(visit_dates)}")
for visit_date in visit_dates[:10]:  # 最初の10件を表示
    print(f"[RESULT] {visit_date}")

# 結果をCSVファイルに保存
df = pd.DataFrame(visit_dates)
df.to_csv('visit_dates.csv', index=False, encoding='utf-8-sig')
print("[INFO] Visit dates have been saved to visit_dates.csv")

[INFO] Scraping page 1: https://www.jalan.net/kankou/spt_10426cd2110076558/kuchikomi/page_1/?screenId=OUW2202&reviewRefineCompanion=all&reviewRefineMonth=all&resultSort=pd
 VisitDate: 2024年11月
 VisitDate: 2024年11月
 VisitDate: 2024年6月
 VisitDate: 2024年11月13日
 VisitDate: 2024年10月18日
 VisitDate: 2024年10月
 VisitDate: 2024年11月
 VisitDate: 2024年11月
 VisitDate: 2024年11月7日
 VisitDate: 2024年10月13日
Waiting for 3.58 seconds before the next request...
[INFO] Scraping page 2: https://www.jalan.net/kankou/spt_10426cd2110076558/kuchikomi/page_2/?screenId=OUW2202&reviewRefineCompanion=all&reviewRefineMonth=all&resultSort=pd
 VisitDate: 2024年10月20日
 VisitDate: 2024年10月17日
 VisitDate: 2024年10月
 VisitDate: 2024年10月
 VisitDate: 2024年9月22日
 VisitDate: 2024年8月
 VisitDate: 2024年8月
 VisitDate: 2024年8月21日
 VisitDate: 2024年8月9日
 VisitDate: 2024年8月3日
Waiting for 3.51 seconds before the next request...
[INFO] Scraping page 3: https://www.jalan.net/kankou/spt_10426cd2110076558/kuchikomi/page_3/?screenId=OUW2202&re